In [1]:
import numpy as np

In [2]:
# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print(len(text))

1115394


In [4]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [5]:
# # get all the character
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [6]:
# in practice, it's better to not restrict the length of the vocabulary to 65
# instead, the vocabularies are sub-words, not fully words, but also not just letters

# tokenize - character mappings
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
# encoder: takes a string, outputs a list of integers
encode = lambda s: [stoi[c] for c in s]
# decoder: takes a list of integers, outputs a string
decode = lambda l: ''.join([itos[i] for i in l])
print(encode("wassup"))
print(decode(encode("wassup")))

[61, 39, 57, 57, 59, 54]
wassup


In [7]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])


torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [8]:
# split data into train and validation sets
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[:n]

note to self: train the model on chunks of the dataset

In [9]:
# hyperparameters
batch_size = 4
block_size = 8
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
eval_iters = 200
n_embd = 16 # number of dimensions for embeddings
n_head = 4
n_layer = 4
dropout = 0.2
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [10]:
torch.manual_seed(1337)
batch_size = 4 # parallel processing of random chunks
block_size = 8 # context used to predict next character

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i: i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)
print('----')

for b in range(batch_size): # batch_dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f'when input is {context.tolist()} then the target: {target}')


inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
when input is [24] then the target: 43
when input is [24, 43] then the target: 58
when input is [24, 43, 58] then the target: 5
when input is [24, 43, 58, 5] then the target: 57
when input is [24, 43, 58, 5, 57] then the target: 1
when input is [24, 43, 58, 5, 57, 1] then the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] then the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] then the target: 39
when input is [44] then the target: 53
when input is [44, 53] then the target: 56
when input is [44, 53, 56] then the target: 1
when input is [44, 53, 56, 1] then the target: 58
when input

In [11]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class Head(nn.Module):
    """ one head of self-attention"""
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x) # (B, T, C)
        q = self.query(x) # (B, T, C)
        
        # compute attention scores ("affinities") 
        wei = q @ k.transpose(-2,-1) * C ** -.5 #(B, T, C) @ (B, C, T) => # (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B, T, C)
        out = wei @ v # (B, T, T) @ (B, T, C) => # (B, T, C)
        
        return out
        

class MultiHeadAttention(nn.Module):
    """multiple heads of self-attention in parallel"""
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedForward(nn.Module):
    """a simple layer followed by non-linearity"""
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout)
        )
    def forward(self, x):
        return self.net(x)
    
class Block(nn.Module):
    """Transformer block: communication followed by computation"""
    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
    
    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        # doing the layer norm before the addition instead of after (pre-norm)  is a slight deviation from the transformer paper
        x = x + self.ffwd(self.ln2(x))
        return x

class BigramLanguageModel(nn.Module):
    
    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)
    
    def forward(self, idx, targets=None):
        B, T = idx.shape
        
        # idx and targets are both (B, T) tensors of integers
        tok_emb = self.token_embedding_table(idx) # (B, T, C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T, C)
        x = tok_emb + pos_emb # (B, T, C)
        x = self.blocks(x) # (B, T, C)
        x = self.ln_f(x)
        logits = self.lm_head(x) # (B, T, vocab_size)
        
        if targets is None:
            loss = None
        else:
            # cross-entropy expect (minibatch, channel) logits so we have to reshape logits to (B*T, C)
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in current context
        for _ in range(max_new_tokens):
            # rop idx to last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get predictions
            logits, loss = self(idx_cond)
            # focus no last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sample to running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T + 1)
        return idx

m = BigramLanguageModel().to(device)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)


print(m.generate(idx = torch.zeros((1,1), dtype=torch.long), max_new_tokens=100)[0].tolist())

torch.Size([32, 65])
tensor(4.4286, grad_fn=<NllLossBackward0>)
[0, 49, 35, 5, 59, 18, 54, 50, 1, 24, 64, 48, 57, 31, 3, 20, 53, 34, 18, 17, 3, 63, 29, 52, 45, 40, 0, 1, 63, 46, 18, 48, 27, 54, 11, 2, 7, 16, 47, 49, 37, 18, 16, 38, 3, 54, 12, 43, 25, 27, 12, 20, 43, 19, 54, 3, 63, 52, 33, 18, 9, 26, 37, 40, 27, 2, 28, 35, 38, 27, 51, 2, 61, 15, 8, 7, 12, 15, 57, 24, 10, 64, 20, 40, 63, 48, 57, 40, 16, 64, 50, 27, 32, 27, 34, 38, 49, 19, 12, 41, 3]


In [12]:
optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate)

In [13]:
@torch.no_grad()
def estimate_loss():
    out = {}
    m.eval()
    for split in ['train', 'eval']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y = get_batch(split)
            logits, loss = m(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    m.train()
    return out

In [14]:
for steps in range(max_iters):
    if steps % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['eval']:.4f}")
    # sample batch of data
    xb, yb = get_batch('train')
    
    # evaluate loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
print(loss.item())

step <built-in function iter>: train loss 4.3588, val loss 4.3662
step <built-in function iter>: train loss 3.3874, val loss 3.3932
step <built-in function iter>: train loss 3.2532, val loss 3.2787
step <built-in function iter>: train loss 3.1658, val loss 3.2195
step <built-in function iter>: train loss 3.0917, val loss 3.0924
step <built-in function iter>: train loss 3.0196, val loss 2.9896
step <built-in function iter>: train loss 2.9067, val loss 2.9284
step <built-in function iter>: train loss 2.8501, val loss 2.8691
step <built-in function iter>: train loss 2.8360, val loss 2.8522
step <built-in function iter>: train loss 2.7825, val loss 2.7851
step <built-in function iter>: train loss 2.7557, val loss 2.7600
step <built-in function iter>: train loss 2.6926, val loss 2.6999
step <built-in function iter>: train loss 2.6879, val loss 2.7012
step <built-in function iter>: train loss 2.6582, val loss 2.6758
step <built-in function iter>: train loss 2.6734, val loss 2.6741
step <buil

In [15]:
print(decode(m.generate(idx = torch.zeros((1,1), dtype=torch.long, device=device), max_new_tokens=500)[0].tolist()))


Wher lilon tithd thowlq:
Whal,
Xon : ka inthe?

Nourd sun andestive and ve ome fmeens
Ame Wom atheded Ind O
Thes ma f,
Ain fag unds kyouts Lnod thnoue berede brethe y, argr lof o yos IN Iut, cow, isht oth norerd spared'as fangt coe too sir theen yosthayplt, tory heth torlory. sithstung nera fo wes thaste dee bom
Hild yoked lexve d noundut tove be vyir n paofgh terpikense hous I teons the fas lyoo!
Bek fl fov
Monowreareng.


FPWE'Wh usthy sond, y?
TKhikth Ho mend Uolo divecaef rorn ecoun-rhel ser


the mathematical trick in self-attention (transformers) 

In [ ]:
B, T, C = 4, 8, 32
x = torch.randn(B, T, C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x) # (B, T, 16)
q = query(x) # (B, T, 16)
wei = q @ k.transpose(-2,-1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)

out = wei @ v

wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1025, 0.8975, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3915, 0.3952, 0.2133, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1635, 0.0163, 0.4267, 0.3935, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2413, 0.4774, 0.0433, 0.0468, 0.1912, 0.0000, 0.0000, 0.0000],
        [0.1654, 0.3547, 0.0711, 0.0682, 0.0819, 0.2586, 0.0000, 0.0000],
        [0.3058, 0.2215, 0.0487, 0.1076, 0.2070, 0.0384, 0.0709, 0.0000],
        [0.3167, 0.0865, 0.0342, 0.0117, 0.1029, 0.1120, 0.2438, 0.0922]],
       grad_fn=<SelectBackward0>)

note how softmax sharpens towards the max value when the variance increases

In [34]:
torch.softmax(torch.tensor([0.1,-0.2,0.3,-0.2,0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [35]:
torch.softmax(torch.tensor([0.1,-0.2,0.3,-0.2,0.5]) * 8, dim=-1)

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

In [ ]:
# solution, dividing by sqrt of head size
torch.softmax(torch.tensor([0.1,-0.2,0.3,-0.2,0.5]) * 8 / 8**0.5, dim=-1)

tensor([0.1489, 0.0637, 0.2621, 0.0637, 0.4615])